In [ ]:
# !pip install datasets

# !pip install simpletransformers

# !pip install --upgrade accelerate

# !pip uninstall -y transformers accelerate
# !pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 13.1 MB/s eta 0:

In [ ]:
from typing import Dict, Any
from datasets import load_dataset, concatenate_datasets
# import evaluate

from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CKPT = 't5-small'
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = T5ForConditionalGeneration.from_pretrained(CKPT)

In [ ]:
# PREPARING DATASET

In [ ]:
"""A large crowd-sourced dataset for developing natural language interfaces for relational databases"""


import json
import os

import datasets


In [ ]:
_DESCRIPTION = """\
A large crowd-sourced dataset for developing natural language interfaces for relational databases
"""

_AGG_OPS = ["", "MAX", "MIN", "COUNT", "SUM", "AVG"]
_COND_OPS = ["=", ">", "<", "OP"]

In [ ]:
def _convert_to_human_readable(sel, agg, columns, conditions):


        rep = f"SELECT {_AGG_OPS[agg]} {columns[sel] if columns is not None else f'col{sel}'} FROM table"

        if conditions:
            rep += " WHERE " + " AND ".join([f"{columns[i]} {_COND_OPS[o]} {v}" for i, o, v in conditions])
        return " ".join(rep.split())

def _generate_examples(main_filepath, tables_filepath):
  """Yields examples."""

  # Build dictionary to table_ids:tables
  with open(tables_filepath, encoding="utf-8") as f:
      tables = [json.loads(line) for line in f]
      id_to_tables = {x["id"]: x for x in tables}
  rows = []
  ids =[]
  with open(main_filepath, encoding="utf-8") as f:
      for idx, line in enumerate(f):
          row = json.loads(line)
          row["table"] = id_to_tables[row["table_id"]]
          del row["table_id"]

          # Handle missing data
          row["table"]["page_title"] = row["table"].get("page_title", "")
          row["table"]["section_title"] = row["table"].get("section_title", "")
          row["table"]["caption"] = row["table"].get("caption", "")
          row["table"]["name"] = row["table"].get("name", "")
          row["table"]["page_id"] = str(row["table"].get("page_id", ""))

          # Fix row types
          row["table"]["rows"] = [[str(e) for e in r] for r in row["table"]["rows"]]

          # Get human-readable version
          row["sql"]["human_readable"] = _convert_to_human_readable(
              row["sql"]["sel"],
              row["sql"]["agg"],
              row["table"]["header"],
              row["sql"]["conds"],
          )

          # Restructure sql->conds
          # - wikiSQL provides a tuple [column_index, operator_index, condition]
          #   as 'condition' can have 2 types (float or str) we convert to dict
          for i in range(len(row["sql"]["conds"])):
              row["sql"]["conds"][i] = {
                  "column_index": row["sql"]["conds"][i][0],
                  "operator_index": row["sql"]["conds"][i][1],
                  "condition": str(row["sql"]["conds"][i][2]),
              }
              rows.append(row)
              ids.append(i)
      return rows


In [ ]:
main_filepath = "/content/drive/MyDrive/wikisqldata/train.jsonl"
tables_filepath = "/content/drive/MyDrive/wikisqldata/train.tables.jsonl"
train_da =_generate_examples(main_filepath, tables_filepath)
main_filepath = "/content/drive/MyDrive/wikisqldata/dev.jsonl"
tables_filepath = "/content/drive/MyDrive/wikisqldata/dev.tables.jsonl"
val_da =_generate_examples(main_filepath, tables_filepath)
main_filepath = "/content/drive/MyDrive/wikisqldata/test.jsonl"
tables_filepath = "/content/drive/MyDrive/wikisqldata/test.tables.jsonl"
test_da =_generate_examples(main_filepath, tables_filepath)

In [ ]:
print(len(train_da))
print(len(val_da))
print(len(test_da))

76729
11545
21846


In [ ]:
import pandas as pd
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=train_da))
validation_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=val_da))
test_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=test_da))

In [ ]:
from datasets.dataset_dict import DatasetDict

In [ ]:
dataset1 = DatasetDict({'train':train_dataset,'validation':validation_dataset,'test':test_dataset})

In [ ]:
dataset2 = DatasetDict({'train':train_dataset,'validation':validation_dataset,'test':test_dataset})

In [ ]:
# Let's format the datasets

In [ ]:


def format_dataset_1(example):
 return {'input': 'translate to SQL: ' + example['question'], 'target': example['sql']['human_readable']}


def format_dataset_2(example):
 return {'input': 'translate to English: ' + example['sql']['human_readable'], 'target': example['question'] }



In [ ]:
ds_1 = dataset1.map(format_dataset_1, remove_columns=dataset1['train'].column_names)

Map:   0%|          | 0/76729 [00:00<?, ? examples/s]

Map:   0%|          | 0/11545 [00:00<?, ? examples/s]

Map:   0%|          | 0/21846 [00:00<?, ? examples/s]

In [ ]:
ds_2 = dataset2.map(format_dataset_2, remove_columns=dataset2['train'].column_names)

Map:   0%|          | 0/76729 [00:00<?, ? examples/s]

Map:   0%|          | 0/11545 [00:00<?, ? examples/s]

Map:   0%|          | 0/21846 [00:00<?, ? examples/s]

In [ ]:
ds_1['train'][0], ds_2['train'][0]

({'input': 'translate to SQL: Tell me what the notes are for South Australia ',
  'target': 'SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA'},
 {'input': 'translate to English: SELECT Notes FROM table WHERE Current slogan = SOUTH AUSTRALIA',
  'target': 'Tell me what the notes are for South Australia '})

In [ ]:
type(ds_1)

datasets.dataset_dict.DatasetDict

In [ ]:
ds_1

DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 76729
    })
    validation: Dataset({
        features: ['input', 'target'],
        num_rows: 11545
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 21846
    })
})

In [ ]:
from datasets import concatenate_datasets

train_data = concatenate_datasets([ds_1['train'],ds_1['validation']]).shuffle(seed=42)
test_data = concatenate_datasets([ds_1['test']]).shuffle(seed=42)

In [ ]:
train_data

Dataset({
    features: ['input', 'target'],
    num_rows: 88274
})

In [ ]:
test_data

Dataset({
    features: ['input', 'target'],
    num_rows: 21846
})

In [ ]:
# Let's calculate the inputs and targets lengths

In [ ]:


# map article and summary len to dict as well as if sample is longer than 512 tokens
def map_to_length(x):
  x["input_len"] = len(tokenizer(x["input"]).input_ids)
  x["input_longer_256"] = int(x["input_len"] > 256)
  x["input_longer_128"] = int(x["input_len"] > 128)
  x["input_longer_64"] = int(x["input_len"] > 64)
  x["out_len"] = len(tokenizer(x["target"]).input_ids)
  x["out_longer_256"] = int(x["out_len"] > 256)
  x["out_longer_128"] = int(x["out_len"] > 128)
  x["out_longer_64"] = int(x["out_len"] > 64)
  return x

sample_size = 10000
data_stats = train_data.select(range(sample_size)).map(map_to_length, num_proc=4)

def compute_and_print_stats(x):
  if len(x["input_len"]) == sample_size:
    print(
        "Input Mean: {}, %-Input > 256:{},  %-Input > 128:{}, %-Input > 64:{} Output Mean:{}, %-Output > 256:{}, %-Output > 128:{}, %-Output > 64:{}".format(
            sum(x["input_len"]) / sample_size,
            sum(x["input_longer_256"]) / sample_size,
            sum(x["input_longer_128"]) / sample_size,
            sum(x["input_longer_64"]) / sample_size,
            sum(x["out_len"]) / sample_size,
            sum(x["out_longer_256"]) / sample_size,
            sum(x["out_longer_128"]) / sample_size,
            sum(x["out_longer_64"]) / sample_size,
        )
    )

output = data_stats.map(
  compute_and_print_stats,
  batched=True,
  batch_size=-1,
)

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Input Mean: 23.889, %-Input > 256:0.0,  %-Input > 128:0.0, %-Input > 64:0.0007 Output Mean:23.5552, %-Output > 256:0.0, %-Output > 128:0.0, %-Output > 64:0.0011


In [ ]:
# As we can see input and output mean are around 20 tokens. So we should not use less than 20 token in both econder and decoder to keep all the info there. For this example we will use 64, but we could use 32.

In [ ]:
# Let's tokenize our data

In [ ]:


# tokenize the examples
def convert_to_features(example_batch):
    input_encodings = tokenizer.batch_encode_plus(example_batch['input'], pad_to_max_length=True, max_length=64)
    target_encodings = tokenizer.batch_encode_plus(example_batch['target'], pad_to_max_length=True, max_length=64)

    encodings = {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids'],
        'decoder_attention_mask': target_encodings['attention_mask']
    }

    return encodings

In [ ]:
train_data = train_data.map(convert_to_features, batched=True, remove_columns=train_data.column_names)
test_data = test_data.map(convert_to_features, batched=True, remove_columns=test_data.column_names)

columns = ['input_ids', 'attention_mask', 'labels', 'decoder_attention_mask']

train_data.set_format(type='torch', columns=columns)
test_data.set_format(type='torch', columns=columns)

Map:   0%|          | 0/88274 [00:00<?, ? examples/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Map:   0%|          | 0/21846 [00:00<?, ? examples/s]

In [ ]:
from transformers import Seq2SeqTrainer
from transformers import Seq2SeqTrainingArguments

In [ ]:
# Set the training arguments

In [ ]:

# set training arguments - Feel free to adapt it
training_args = Seq2SeqTrainingArguments(
    output_dir="/content/t5-small-finetuned-wikisql-sql-nl-nl-sql",
    per_device_train_batch_size=16,
    num_train_epochs=1,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    evaluation_strategy="epoch",
    do_train=True,
    do_eval=True,
    logging_steps=500,
    save_strategy="epoch",
    #save_steps=1000,
    #eval_steps=1000,
    overwrite_output_dir=True,
    save_total_limit=3,
    load_best_model_at_end=True
    #push_to_hub=True
    #fp16=True,
)


In [ ]:
# Let's define a metric to track our training progress

! pip install -q sacrebleu

In [ ]:

import numpy as np
from datasets import load_metric

metric = load_metric("sacrebleu")

In [ ]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
# Let's create a Trainer instance

# instantiate trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=test_data,
)

In [ ]:
# # Sanity check
# trainer.evaluate(eval_dataset=test_data.select(range(100)))

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.save_model("/content/drive/MyDrive/Sandeep_t5_base_wikisql")

In [ ]:
trainer.create_model_card()

In [ ]:
# TEST THE MODEL

In [ ]:
CKPT = '/content/drive/MyDrive/Sandeep_t5_base_wikisql'
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained(CKPT)

In [ ]:
def translate(text):
    inputs = tokenizer(text, padding='longest', max_length=64, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=64)

    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
text = "translate to SQL: How many model with BERT architecture are in the HuggingFace Hub?"
translate(text)


In [ ]:
text = "translate to English: SELECT COUNT Model FROM table WHERE Architecture = RoBERTa AND creator = Manuel Romero"
translate(text)


In [ ]:
from datasets import load_dataset
test_data = load_dataset('wikisql', split='test')

In [ ]:
for i in range(0,50,1):
  if i % 2 == 0:
    print('translate to SQL: ' + test_data[i]['question'])
    print('Predict.: ' + translate('translate to SQL: ' + test_data[i]['question']))
    print('Expected: ' + test_data[i]['sql']['human_readable'])
  else:
    print('translate to English: ' + test_data[i]['sql']['human_readable'])
    print('Predict.: ' + translate('translate to English: ' + test_data[i]['sql']['human_readable']))
    print('Expected: ' + test_data[i]['question'])
  print('=================================\n')

In [ ]:
# https://github.com/mrm8488/shared_colab_notebooks/blob/master/T5_wikiSQL_multitask_with_HF_transformers.ipynb